# Share Price Prediction using RNN

In [55]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np

In [56]:
sp = pd.read_csv("/mnt/New_Volume/Work_From_Home/Data/SBIN.csv")
sp.head()

,Date,series,OPEN,HIGH,LOW,PREV. CLOSE,ltp,close,vwap,52W H,52W L,VOLUME,VALUE,No of trades
0,01-Apr-21,EQ,367.70,371.9,363.10,364.30,370.95,370.65,367.07,427.7,149.45,31883453,"11,703,603,342.10",201211
1,05-Apr-21,EQ,367.50,369.2,349.05,370.65,354.00,353.55,355.21,427.7,149.45,51743981,"18,379,797,141.10",382160
2,06-Apr-21,EQ,355.70,357.0,349.30,353.55,350.95,350.55,353.10,427.7,149.45,44147709,"15,588,499,051.85",273760
3,07-Apr-21,EQ,351.25,363.2,347.60,350.55,358.20,358.55,358.22,427.7,149.45,48023602,"17,203,160,038.90",323113
4,08-Apr-21,EQ,361.00,363.3,354.30,358.55,355.95,355.60,358.65,427.7,149.45,33370259,"11,968,110,091.25",221845


In [57]:
sp.isna().sum()

Date             0
series           0
OPEN             0
HIGH             0
LOW              0
PREV. CLOSE      0
ltp              0
close            0
vwap             0
52W H            0
52W L            0
VOLUME           0
VALUE            0
No of trades     0
dtype: int64

In [58]:
sp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           39 non-null     object 
 1   series         39 non-null     object 
 2   OPEN           39 non-null     float64
 3   HIGH           39 non-null     float64
 4   LOW            39 non-null     float64
 5   PREV. CLOSE    39 non-null     float64
 6   ltp            39 non-null     float64
 7   close          39 non-null     float64
 8   vwap           39 non-null     float64
 9   52W H          39 non-null     float64
 10  52W L          39 non-null     float64
 11  VOLUME         39 non-null     int64  
 12  VALUE          39 non-null     object 
 13  No of trades   39 non-null     int64  
dtypes: float64(9), int64(2), object(3)
memory usage: 4.4+ KB


# Target is Closing Price

In [59]:
sp

,Date,series,OPEN,HIGH,LOW,PREV. CLOSE,ltp,close,vwap,52W H,52W L,VOLUME,VALUE,No of trades
0,01-Apr-21,EQ,367.70,371.90,363.10,364.30,370.95,370.65,367.07,427.70,149.45,31883453,"11,703,603,342.10",201211
1,05-Apr-21,EQ,367.50,369.20,349.05,370.65,354.00,353.55,355.21,427.70,149.45,51743981,"18,379,797,141.10",382160
2,06-Apr-21,EQ,355.70,357.00,349.30,353.55,350.95,350.55,353.10,427.70,149.45,44147709,"15,588,499,051.85",273760
3,07-Apr-21,EQ,351.25,363.20,347.60,350.55,358.20,358.55,358.22,427.70,149.45,48023602,"17,203,160,038.90",323113
4,08-Apr-21,EQ,361.00,363.30,354.30,358.55,355.95,355.60,358.65,427.70,149.45,33370259,"11,968,110,091.25",221845
5,09-Apr-21,EQ,354.40,364.50,351.20,355.60,352.95,353.00,357.30,427.70,149.45,46473100,"16,604,696,682.20",303846
6,12-Apr-21,EQ,344.00,344.00,322.55,353.00,330.70,328.85,329.89,427.70,149.45,75501713,"24,907,319,137.55",579319
7,13-Apr-21,EQ,332.00,342.25,330.50,328.85,341.25,341.00,336.40,427.70,149.45,49773360,"16,743,865,802.60",319183
8,15-Apr-21,EQ,342.10,347.55,336.10,341.00,342.55,342.70,341.70,427.70,149.45,52992349,"18,107,434,663.30",325566
9,16-Apr-21,EQ,343.00,345.50,338.65,342.70,339.55,339.90,342.01,427.70,149.45,37766793,"12,916,560,525.30",237384


In [60]:
sp.columns

Index(['Date ', 'series ', 'OPEN ', 'HIGH ', 'LOW ', 'PREV. CLOSE ', 'ltp ',
       'close ', 'vwap ', '52W H ', '52W L ', 'VOLUME ', 'VALUE ',
       'No of trades '],
      dtype='object')

In [61]:
sp.drop(labels = ['Date ','series ','52W H ', '52W L ', 'VOLUME ', 'VALUE '], axis = 1, inplace = True)
sp

,OPEN,HIGH,LOW,PREV. CLOSE,ltp,close,vwap,No of trades
0,367.70,371.90,363.10,364.30,370.95,370.65,367.07,201211
1,367.50,369.20,349.05,370.65,354.00,353.55,355.21,382160
2,355.70,357.00,349.30,353.55,350.95,350.55,353.10,273760
3,351.25,363.20,347.60,350.55,358.20,358.55,358.22,323113
4,361.00,363.30,354.30,358.55,355.95,355.60,358.65,221845
5,354.40,364.50,351.20,355.60,352.95,353.00,357.30,303846
6,344.00,344.00,322.55,353.00,330.70,328.85,329.89,579319
7,332.00,342.25,330.50,328.85,341.25,341.00,336.40,319183
8,342.10,347.55,336.10,341.00,342.55,342.70,341.70,325566
9,343.00,345.50,338.65,342.70,339.55,339.90,342.01,237384


# Splitting the data into x and y

In [137]:
y = sp['close ']
x = sp.drop('close ', axis  = 1)

In [138]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# x = scaler.fit_transform(x)

In [139]:
x = np.array(x)
# x

In [140]:
x = x[:,np.newaxis,:]

In [141]:
x

array([[[3.677000e+02, 3.719000e+02, 3.631000e+02, 3.643000e+02,
         3.709500e+02, 3.670700e+02, 2.012110e+05]],

       [[3.675000e+02, 3.692000e+02, 3.490500e+02, 3.706500e+02,
         3.540000e+02, 3.552100e+02, 3.821600e+05]],

       [[3.557000e+02, 3.570000e+02, 3.493000e+02, 3.535500e+02,
         3.509500e+02, 3.531000e+02, 2.737600e+05]],

       [[3.512500e+02, 3.632000e+02, 3.476000e+02, 3.505500e+02,
         3.582000e+02, 3.582200e+02, 3.231130e+05]],

       [[3.610000e+02, 3.633000e+02, 3.543000e+02, 3.585500e+02,
         3.559500e+02, 3.586500e+02, 2.218450e+05]],

       [[3.544000e+02, 3.645000e+02, 3.512000e+02, 3.556000e+02,
         3.529500e+02, 3.573000e+02, 3.038460e+05]],

       [[3.440000e+02, 3.440000e+02, 3.225500e+02, 3.530000e+02,
         3.307000e+02, 3.298900e+02, 5.793190e+05]],

       [[3.320000e+02, 3.422500e+02, 3.305000e+02, 3.288500e+02,
         3.412500e+02, 3.364000e+02, 3.191830e+05]],

       [[3.421000e+02, 3.475500e+02, 3.361000e+0

In [142]:
x.shape

(39, 1, 7)

In [143]:
np.random.seed(42)
tf.random.set_seed(42)
keras.backend.clear_session()
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape = [1,7])
])

In [144]:
optimizer = keras.optimizers.Adam(learning_rate= 0.005)
model.compile(loss = 'mse', optimizer = optimizer)

In [145]:
model.fit(x,y, validation_split = 0.2,epochs = 50 )

Epoch 1/50
1/1 [==============================] - 1s 901ms/step - loss: 124890.5078 - val_loss: 169011.6250
Epoch 2/50
1/1 [==============================] - 0s 44ms/step - loss: 124890.5156 - val_loss: 169011.6250
Epoch 3/50
1/1 [==============================] - 0s 55ms/step - loss: 124890.5156 - val_loss: 169011.6250
Epoch 4/50
1/1 [==============================] - 0s 56ms/step - loss: 124890.5156 - val_loss: 169011.6250
Epoch 5/50
1/1 [==============================] - 0s 54ms/step - loss: 124890.5156 - val_loss: 169011.6250
Epoch 6/50
1/1 [==============================] - 0s 55ms/step - loss: 124890.5078 - val_loss: 169011.6250
Epoch 7/50
1/1 [==============================] - 0s 53ms/step - loss: 124890.5156 - val_loss: 169011.6250
Epoch 8/50
1/1 [==============================] - 0s 64ms/step - loss: 124890.5078 - val_loss: 169011.6250
Epoch 9/50
1/1 [==============================] - 0s 56ms/step - loss: 124890.5078 - val_loss: 169011.6250
Epoch 10/50
1/1 [===================

In [146]:
model.predict([[[4.210000e+02, 4.260000e+02, 4.180000e+02, 4.220500e+02,
         4.245000e+02, 4.229000e+02, 2.750820e+05]]])

array([[1.]], dtype=float32)

In [119]:
test = np.arange(1,7).reshape(3,2)
test

array([[1, 2],
       [3, 4],
       [5, 6]])

In [120]:
test.ndim

2

In [121]:
test_1 = test[...,np.newaxis]
test_1

array([[[1],
        [2]],

       [[3],
        [4]],

       [[5],
        [6]]])

In [122]:
test_2 = test[:,np.newaxis,:]
test_2

array([[[1, 2]],

       [[3, 4]],

       [[5, 6]]])

In [135]:
test_3 = test[np.newaxis,...]

In [136]:
test_3

array([[[1, 2],
        [3, 4],
        [5, 6]]])